In [1]:
from qkeras import *
from tensorflow.keras.layers import Input, AveragePooling2D, Flatten, Softmax, Add, ZeroPadding2D, MaxPooling2D
import numpy as np
from collections import namedtuple
import pickle
import math
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import plot_model
from tensorflow.keras.utils import to_categorical
from qkeras.utils import model_save_quantized_weights

c:\ProgramData\Miniconda3\envs\qkeras\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\ProgramData\Miniconda3\envs\qkeras\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\ProgramData\Miniconda3\envs\qkeras\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
def load_data(num_classes=10, subtract_pixel_mean=True):
    """
    Load CIFAR10 data and normalize
    """
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()

    # input image dimensions.
    input_shape = x_train.shape[1:]

    # normalize data.
    x_train = x_train.astype('float32') / 128.0 - 1.0
    x_test = x_test.astype('float32') / 128.0 - 1.0

    # if subtract pixel mean is enabled
    if subtract_pixel_mean:
        x_train_mean = np.mean(x_train, axis=0)
        x_train -= x_train_mean
        x_test -= x_train_mean

    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')
    print('y_train shape:', y_train.shape)

    # convert class vectors to binary class matrices,
    # i.e., one hot encodings
    y_train = to_categorical(y_train, num_classes)
    y_test = to_categorical(y_test, num_classes)

    return x_train, y_train, x_test, y_test


In [3]:
x_train, y_train, x_test, y_test = load_data(10, False)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
y_train shape: (50000, 1)


In [4]:
input_shape = x_train.shape[1:-1] + (3,)
np.random.seed(1)

a_0 = 'quantized_relu(8,0,negative_slope=0.125)'
a_1 = 'quantized_relu(8,1,negative_slope=0.125)'
a_2 = 'quantized_relu(8,2,negative_slope=0.125)'
a_3 = 'quantized_relu(8,3,negative_slope=0.125)'

q_0 = 'quantized_bits(8,0,False,True,1)'
q_1 = 'quantized_bits(8,1,False,True,1)'
q_2 = 'quantized_bits(8,2,False,True,1)'
q_3 = 'quantized_bits(8,3,False,True,1)'

q_t = 'quantized_bits(8,0,False,True,1)'

np.random.seed(42)
#preamble = './drive/MyDrive/resnet/'
preamble = ''
USE_BIAS = True

In [5]:
class Bundle(tf.keras.Model):
    def __init__(self, 
                 core,             # dict, Mandaroty: parameters for conv/dense layer, act can be quantization or relu
                 add=None,         # dict, Mandatory if x1 is not None in call(), else ignored
                 pool=None,        # dict, Optional: can only be max or avg
                 flatten=False,    # Optional: set to True to flatten the outputs
                 softmax=False,    # Optional: set to Ture to include floating point softmax layer
                 **kwargs):

        super(Bundle, self).__init__()
        
        self.core = core
        self.add = add
        self.pool = pool
        self.flatten = flatten
        self.softmax = softmax
        self.inp = {'tensor':None, 'int': None, 'bits':None, 'frac': None}
        self.out = {'tensor':None, 'int': None, 'bits':None, 'frac': None}
        self.proc = {'tensor':None, 'int': None, 'bits':None, 'frac': None}
        self.w = {'tensor':None, 'int': None, 'bits':None, 'frac': None}
        self.b = {'tensor':None, 'int': None, 'bits':None, 'frac': None}

        # Store reference to bundle object here, not just a idx number
        self.prev_bundle = None
        self.add_bundle = None

        def extract_act(signature):
            ilayer = QActivation(signature)
            d = ilayer.quantizer.get_config()
            sign_bit = d['keep_negative'] if 'keep_negative' in d else (d['negative_slope'] !=0 if 'negative_slope' in d else (0))
            int_bit = d['integer'] if 'integer' in d else 0
            frac = d['bits']-int_bit-sign_bit

            if isinstance(ilayer.quantizer, quantized_bits):
                return { 'layer':ilayer, 'type':'quant', 'bits':d['bits'], 'frac':frac}
            elif 'relu' in str(ilayer.quantizer.__class__) and ilayer.quantizer.negative_slope != 0:
                return { 'layer':ilayer, 'type':'relu', 'slope':ilayer.quantizer.negative_slope, 'bits':d['bits'], 'frac':frac}
            else:
                raise Exception("Only leaky_relu (relu with negative_slope > 0) is suppported!")

        '''
        CORE LAYER
        '''
        if core['type'] == 'conv':
            for i in ['filters', 'kernel_size', 'strides', 'padding', 'kernel_quantizer', 'bias_quantizer', 'use_bias', 'act_str']:
                assert i in core, f"'{i}' must be provided for conv"

            self.core['layer'] = QConv2DBatchnorm(
                filters=self.core['filters'], kernel_size=self.core['kernel_size'], strides=self.core['strides'],
                padding=self.core['padding'], kernel_quantizer=self.core['kernel_quantizer'], 
                bias_quantizer=self.core['bias_quantizer'], use_bias=self.core['use_bias'])
        
        else:
            for i in ['units', 'kernel_quantizer', 'bias_quantizer', 'use_bias', 'act_str']:
                assert i in self.core, f"'{i}' must be provided for dense"
            
            self.core['layer'] = QDense(
                units=self.core['units'], kernel_quantizer=self.core['kernel_quantizer'],
                bias_quantizer=self.core['bias_quantizer'], use_bias=self.core['use_bias'])

        '''
        CORE ACT LAYER
        '''
        self.core['act'] = extract_act(core['act_str'])
        self.out['frac'], self.out['bits'] = self.core['act']['frac'], self.core['act']['bits']

        '''
        ACT ADD LAYER
        '''
        if self.add is not None:
            self.add['act'] = extract_act(add['act_str'])
            self.out['frac'], self.out['bits'] = self.add['act']['frac'], self.add['act']['bits']

        '''
        POOL LAYER
        '''
        if pool:
            for i in ['type', 'size', 'strides', 'padding']:
                assert i in pool, f"'{i}' must be provided for pool"

            if pool['type'] == 'max':
                self.pool_layer = MaxPooling2D(self.pool['size'], strides=self.pool['strides'], padding=self.pool['padding'])
            elif pool['type'] == 'avg':
                self.pool_layer = QAveragePooling2D(self.pool['size'], strides=self.pool['strides'], padding=self.pool['padding'])
            else:
                raise Exception(self.pool['type'], "only avg or max pool is supported for now")
            
            self.pool['act'] = extract_act(self.pool['act_str'])
            self.out['frac'], self.out['bits'] = self.pool['act']['frac'], self.pool['act']['bits']
        else:
            self.pool_layer = None

        '''
        FLATTEN & SOFTMAX LAYERS
        '''
        self.flatten_layer = Flatten() if self.flatten else None

        self.softmax = softmax
        self.softmax_layer = Activation("softmax") if self.softmax else None
        if softmax:
            self.out['frac'], self.out['bits'] = 0, 1


    # functions for training
    def call(self, x, x_1=None):
        if hasattr(x, "bundle"):
            self.prev_bundle = x.bundle
            self.idx = self.prev_bundle.idx + 1
        else:
            self.prev_bundle = None
            self.idx = 0

        self.inp['tensor'] = x

        x = self.core['layer'](x)
        x = self.core['act']['layer'](x)
        self.core['tensor'] = x

        if x_1 is not None:
            if hasattr(x_1, "bundle"):
                self.add['bundle'] = x_1.bundle
            else:
                self.add['bundle'] = None
            x = Add()([x, x_1])
            x = self.add['act']['layer'](x)
            self.add['tensor'] = x
        if self.pool_layer:
            x = self.pool_layer(x)
            x = self.pool['act']['layer'](x)
            self.pool['tensor'] = x
        if self.flatten_layer:
            x = self.flatten_layer(x)
        if self.softmax_layer:
            x = self.softmax_layer(x)

        self.out['tensor'] = x
        x.bundle = self
        return x

    # functions to be prepared for exportation
    def load_weight_bias(self):
        k_tensor = self.core['layer'].get_folded_weights()[0] if isinstance(self.core['layer'], QConv2DBatchnorm) else self.core['layer'].kernel
        k = self.core['layer'].kernel_quantizer_internal(k_tensor).numpy()
        k_config = self.core['layer'].kernel_quantizer_internal.get_config()

        k_frac = k_config['bits']-k_config['integer']-k_config['keep_negative']
        k_int = k * 2**k_frac
        assert (k_int == k_int.astype(int)).all(), f"Weights failed integer test for bundle {self.idx}"
        k_int = k_int.astype(int)
        self.w = {'tensor':k_tensor, 'int': k_int, 'bits':k_config['bits'], 'frac':k_frac}

        if (self.core['type'] == 'conv' and self.core['use_bias']) or (self.core['type'] == 'dense' and self.core['use_bias']):
            b_tensor = self.core['layer'].get_folded_weights()[1] if isinstance(self.core['layer'], QConv2DBatchnorm) else self.core['layer'].bias
            b = self.core['layer'].bias_quantizer_internal(b_tensor).numpy()
            b_config = self.core['layer'].bias_quantizer_internal.get_config()
            b_frac = b_config['bits']-b_config['integer']-b_config['keep_negative']
            b_int = b * 2**b_frac
            assert (b_int == b_int.astype(int)).all(), f"Bias failed integer test for bundle {self.idx}"
            b_int = b_int.astype(int)
            self.b = {'tensor':b_tensor, 'int':b_int, 'bits':b_config['bits'], 'frac':b_frac}


    def process(self, inp = None):
        
        ''' Integer test for output '''
        self.out['int'] = self.out['tensor'].numpy() * 2**self.out['frac']
        if self.softmax is None:
            assert (self.out['int'] == self.out['int'].astype(int)).all(), f"Output tensor of bundle {self.idx} is not a fixed point"
            self.out['int'] = self.out['int'].astype(int)

        if inp is not None: # independant mode
            self.inp = inp
        else: # chained mode
            # ToDo: do not rely on external(global) variables!
            self.inp = self.prev_bundle.out
            assert self.idx > 0, "input must be provided manually for the first bundle"

        self.load_weight_bias()
        x = self.inp['int'].astype(np.int32)
        w = self.w['int'].astype(np.int32)

        if self.core['type'] == 'conv':
            self.proc['int'] = tf.keras.backend.conv2d(x, w, padding='same').numpy()
        else:
            self.proc['int'] = x @ w

        self.post_process()


    def post_process(self):
        
        clog2_add = int(np.ceil(np.log2(np.prod(self.w['int'].shape[:-1]))))
        self.proc['bits'] = self.inp['bits'] + self.w['bits'] + clog2_add
        self.proc['frac'] = self.inp['frac'] + self.w['frac']

        if self.b is not None:
            self.proc['int'] += self.b['int'] * 2** (self.proc['frac'] - self.b['frac'])


        if 'strides' in self.core and self.core['strides'] != (1,1):
            SH, SW = self.core['strides']
            N, XH, XW, C = self.proc['int'].shape
            YH, YW = XH//SH, XW//SW
            self.proc['int'] = self.proc['int'].reshape(N, YH, SH, YW, SW, C)
            ind = -1 if self.w['int'].shape[0] > 1 else 0
            self.proc['int'] = self.proc['int'][:,:,ind,:,ind,:]

        def apply_act(act_dict):
            x = self.proc['int'].astype(np.float32)
            frac, bits = act_dict['frac'], act_dict['bits']

            if act_dict['type'] == 'quant':
                x *= 2**(frac-self.proc['frac'])
                x = np.around(x)
                x = np.clip(x, -2**(bits-1), 2**(bits-1)-1).astype(int)

            elif act_dict['type'] == 'relu':
                x *= 2**(frac-self.proc['frac'])
                x = np.clip(x, -2**(bits-1), 2**(bits-1)-1)
                x = np.maximum(x * act_dict['slope'], x)
                x = np.around(x)
                x = np.clip(x,-2**(bits-1), 2**(bits-1)-1).astype(int)
            else:
                raise Exception('Only relu is supported yet')

            self.proc['int'], self.proc['bits'], self.proc['frac'] = x, bits, frac

        apply_act(self.core['act'])
        assert np.all(self.proc['int'] == self.core['tensor'].numpy() * 2**self.proc['frac']), f"Core + act output of bundle {self.idx} is not fixed point"


        if self.add is not None:
            a = self.add['bundle']
            out_frac_add, out_bits_add = max(self.proc['frac'], a.out['frac']), max(self.proc['bits'], a.out['bits'])

            a_arr_cast = a.out['int'] * 2** (out_frac_add - a.out['frac'])
            out_arr_cast = self.proc['int'] * 2 **(out_frac_add - self.proc['frac'])

            self.proc['int'] = out_arr_cast.astype(np.int64) + a_arr_cast.astype(np.int64)
            self.proc['bits'], self.proc['frac'] = out_bits_add, out_frac_add
            apply_act(self.add['act'])

            assert np.all(self.proc['int'] == self.add['tensor'].numpy() * 2**self.proc['frac']), f"Add + act output of bundle {self.idx} is not a fixed point"

        if self.pool_layer:
            if self.pool['type'] == 'max':
                pStride = self.pool['strides']
                pSize = self.pool['size']

                def findMax(InArray, p, q):
                    results = np.zeros((InArray.shape[0], InArray.shape[3]))
                    results -= math.inf
                    for i in range(p, p+pSize[0]):
                        for j in range(q, q+pSize[1]):
                            if i >=0 and j>=0 and i < InArray.shape[1] and j < InArray.shape[2]:
                                cand = InArray[:,i,j,:]
                                results = np.maximum(results, cand)
                    return results

                def HotFixMaxPool2D(InArray):
                    if pStride[0]!=pStride[1] or pSize[0]!=pSize[1]:
                        raise Exception('Only square stride and size is supported')
                    if pSize[0]/2 == 0:
                        raise Exception('Maxpool size should be odd')

                    pad = (pSize[0]-1)//2

                    inShape = InArray.shape
                    assert len(inShape) == 4
                    OutArray = np.zeros((inShape[0], inShape[1]//pStride[0], inShape[2]//pStride[1], inShape[3]))
                    # Start point, should include pad
                    st_p, st_q = -pad, -pad

                    for i in range(OutArray.shape[1]):
                        for j in range(OutArray.shape[2]):
                            p, q = st_p + i*pStride[0] + pStride[0]-1, st_q + j*pStride[1] + pStride[1]-1
                            OutArray[:,i,j,:] = findMax(InArray, p, q)

                    return OutArray

                self.proc['int'] = HotFixMaxPool2D(self.proc['int']).astype(int)

            elif self.pool['type'] == 'avg':
                assert self.pool['size'] == self.pool['strides']
                KH, KW = self.pool['size']
                N, H, W, C = self.proc['int'].shape
                self.proc['int'] = self.proc['int'].reshape(N, H//KH, KH, W//KW, KW, C).mean(axis=(2,4))
                # NO need for clipping, as act_pool in place!
                apply_act(self.pool['act'])
            assert np.all(self.proc['int'] == self.pool['tensor'].numpy() * 2**self.proc['frac']), f"Pool + act output of bundle {self.idx} is not a fixed point"

        if self.flatten:
            self.proc['int'] = self.proc['int'].reshape(self.proc['int'].shape[0],-1)


        if self.softmax:
            self.proc['int'] = self.proc['int'] / 2**self.proc['frac']
            exp = np.exp(self.proc['int'] - self.proc['int'].max())
            self.proc['int'] = exp/np.sum(exp, axis=1)[0]
            assert np.all(np.argmax(self.out['int'], axis=-1) == np.argmax(self.proc['int'], axis=-1))
        else:
            assert np.all(self.proc['int'] == self.out['int']), f"Overall output of bundle {self.idx} is not a fixed point"

        return

class BModel(tf.keras.Model):
    def __init__(self,
                 **kwargs):
        super(BModel, self).__init__()


'''
Build Model
'''

x = x_in = Input(input_shape, name='input')
x = QActivation(q_0)(x)

x = x1 = Bundle(
    core= {'type':'conv', 'filters':64, 'kernel_size':(7,7), 'strides':(2,2), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS, 'act_str':a_0},
    pool= {'type':'max', 'size':(3,3), 'strides':(1,1), 'padding':'same', 'act_str': q_0}
    )(x)

# block 0
x = Bundle(
    core= {'type':'conv', 'filters':64, 'kernel_size':(3,3), 'strides':(1,1), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS, 'act_str':a_0}
    )(x)


x = x1 = Bundle(
    core= {'type':'conv', 'filters':64, 'kernel_size':(3,3), 'strides':(1,1), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS, 'act_str':q_1},
    add= {'act_str': a_0}
    )(x, x1)

x = Bundle(
    core= {'type':'conv', 'filters':64, 'kernel_size':(3,3), 'strides':(1,1), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS, 'act_str':a_0}
    )(x)
x = x1 = Bundle(
    core= {'type':'conv', 'filters':64, 'kernel_size':(3,3), 'strides':(1,1), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS, 'act_str':q_1}, 
    add= {'act_str': a_1}
    )(x, x1)

# block 1
x1 = Bundle(
    core= {'type':'conv', 'filters':128, 'kernel_size':(3,3), 'strides':(2,2), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS, 'act_str':a_1}
    )(x1)
x1 = Bundle(
    core= {'type':'conv', 'filters':128, 'kernel_size':(3,3), 'strides':(1,1), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS, 'act_str':q_2}
    )(x1)
x = x1 = Bundle(
    core= {'type':'conv', 'filters':128, 'kernel_size':(1,1), 'strides':(2,2), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS, 'act_str':q_2},
    add={'act_str':a_2}
    )(x, x1)

x = Bundle(
    core= {'type':'conv', 'filters':128, 'kernel_size':(3,3), 'strides':(1,1), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS, 'act_str':a_1}
    )(x)
x = x1 = Bundle(
    core= {'type':'conv', 'filters':128, 'kernel_size':(1,1), 'strides':(1,1), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS, 'act_str':q_2},
    add={'act_str':a_2}
    )(x, x1)

#block 2
x1 = Bundle(
    core= {'type':'conv', 'filters':256, 'kernel_size':(3,3), 'strides':(2,2), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS, 'act_str':a_1}
    )(x1)
x1 = Bundle(
    core= {'type':'conv', 'filters':256, 'kernel_size':(3,3), 'strides':(1,1), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS, 'act_str':q_2}
    )(x1)
x = x1 = Bundle(
    core = {'type':'conv', 'filters':256, 'kernel_size':(1,1), 'strides':(2,2), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS, 'act_str':q_3},
    add= {'act_str':a_3}
    )(x, x1)

x = Bundle(
    core = {'type':'conv', 'filters':256, 'kernel_size':(3,3), 'strides':(1,1), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS, 'act_str':a_2}
    )(x)
x = x1 = Bundle(
    core = {'type':'conv', 'filters':256, 'kernel_size':(1,1), 'strides':(1,1), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS, 'act_str':q_3},
    add= {'act_str':a_3}
    )(x, x1)

#block 3
x1 = Bundle(
    core={'type':'conv', 'filters':512, 'kernel_size':(3,3), 'strides':(2,2), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS, 'act_str':a_2}
    )(x1)
x1 = Bundle(
    core= {'type':'conv', 'filters':512, 'kernel_size':(3,3), 'strides':(1,1), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS, 'act_str':q_3}
    )(x1)
x = x1 = Bundle(
    core= {'type':'conv', 'filters':512, 'kernel_size':(1,1), 'strides':(2,2), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS, 'act_str':q_3}, 
    add= {'act_str':a_3}
    )(x, x1)

x = Bundle(
    core= {'type':'conv', 'filters':512, 'kernel_size':(3,3), 'strides':(1,1), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS, 'act_str':a_2}
    )(x)
x = Bundle(
    core= {'type':'conv', 'filters':512, 'kernel_size':(1,1), 'strides':(1,1), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS, 'act_str':q_3}, 
    add= {'act_str':a_3},
    pool= {'type':'avg', 'size':(2,2), 'strides':(2,2), 'padding':'valid', 'act_str': q_3},
    flatten=True
    )(x, x1)

x = Bundle(
    core= {'type':'dense', 'units':10, 'kernel_quantizer':q_2, 'bias_quantizer':q_2, 'use_bias':USE_BIAS, 'act_str': q_3}, 
    softmax=True)(x)

model = Model(inputs=x_in, outputs=x)
print(model.summary(expand_nested=True))


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 q_activation (QActivation)     (None, 32, 32, 3)    0           ['input[0][0]']                  
                                                                                                  
 bundle (Bundle)                (None, 16, 16, 64)   9729        ['q_activation[0][0]']           
|¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯|
| q_activation_1 (QActivation)  multiple            0           []                               |
|                                                                                             

In [6]:
def lr_schedule(epoch):
    """
  Bundles_pre_trainearning Rate Schedule
    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.
    # Arguments
        epoch (int): The number of epochs
    # Returns
        lr (float32): learning rate
    """
    # initial_lr = 1e-4
    # lr_decay = 0.99
    # lr = initial_lr * (lr_decay ** epoch)
    lr = 1e-3 # default 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 150:
        lr *= 1e-2
    elif epoch > 100:
        lr *= 1e-1
    elif epoch > 50:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

preamble = ''
model_file_path = preamble+'resnet18.h5'
checkpoint = ModelCheckpoint(filepath=model_file_path,
                                     monitor='val_acc',
                                     verbose=1,
                                     save_best_only=True)
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                                       cooldown=0,
                                       patience=5,
                                       min_lr=0.5e-6)
lr_scheduler = LearningRateScheduler(lr_schedule)

callbacks = [checkpoint, lr_reducer, lr_scheduler]

NB_EPOCH = 200
BATCH_SIZE = 256
VERBOSE = 1
VALIDATION_SPLIT = 0.1
RELU_NEG_SLOPE = 0.125

model.compile(loss='categorical_crossentropy',
            optimizer=Adam(learning_rate=lr_schedule(0)), metrics=['acc'])

# model.fit(x_train, y_train,
#                     batch_size=BATCH_SIZE,
#                     epochs=NB_EPOCH,
#                     validation_data=(x_test, y_test),
#                     shuffle=True,
#                     callbacks=callbacks)

Learning rate:  0.001


In [7]:
XN = 4
x = np.random.randn(XN, *model.input.shape[1:])
x = np.clip(x, -1.0, 1.0)

inp_act_model = Model(inputs=model.input, outputs=model.layers[1].output)
inp ={ 'tensor': inp_act_model(x, training=False), 'bits':8, 'frac':7}
inp['int'] = inp['tensor'].numpy() * 2**inp['frac']

y = model(x)

model.layers[2].process(inp)
for layer in model.layers[3:]:
    layer.process()